In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import requests
import time
import json

In [2]:
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}  # 开启 performance 日志

chrome_options = Options()
chrome_options.add_argument('--headless')  # 无头模式，不打开浏览器窗口
driver = webdriver.Chrome(desired_capabilities=caps, options=chrome_options)

TypeError: WebDriver.__init__() got an unexpected keyword argument 'desired_capabilities'

In [37]:
# 设置浏览器的 DevTools 协议
chrome_options = Options()
chrome_options.add_argument('--headless')  # 无头模式，不打开浏览器窗口
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})  # 开启 performance 日志
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

In [38]:
url = 'https://www.homedepot.com/p/CARRO-Everest-52-in-Dimmable-LED-Indoor-Outdoor-Black-Smart-Ceiling-Fan-Light-and-Remote-Works-with-Alexa-Google-Home-Siri-NS525E-L12-B3/319929479'

In [40]:
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
time.sleep(5)  # 等待页面加载完成
# 设置初始高度
scroll_pause_time = 2  # 每次滚动后等待时间（秒）
scroll_increment = 300  # 每次滚动的距离（像素）
current_scroll_position = 0
new_scroll_position = scroll_increment

# 慢慢滚动页面直到到底部
while True:
    driver.execute_script(f"window.scrollTo({current_scroll_position}, {new_scroll_position});")
    
    # 等待页面加载
    time.sleep(scroll_pause_time)
    
    # 更新滚动位置
    current_scroll_position = new_scroll_position
    new_scroll_position += scroll_increment

    # 获取当前的页面高度
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    
    # 如果滚动到底部则退出
    if current_scroll_position >= scroll_height:
        break


time.sleep(5)

In [42]:
logs = driver.get_log('performance')

In [43]:
logs

[{'level': 'INFO',
  'message': '{"message":{"method":"Page.frameStartedLoading","params":{"frameId":"FD2E2C2E7E3F2C0209F63FC0F336E34C"}},"webview":"FD2E2C2E7E3F2C0209F63FC0F336E34C"}',
  'timestamp': 1726508015371},
 {'level': 'INFO',
  'message': '{"message":{"method":"Network.requestWillBeSent","params":{"documentURL":"https://www.homedepot.com/p/CARRO-Everest-52-in-Dimmable-LED-Indoor-Outdoor-Black-Smart-Ceiling-Fan-Light-and-Remote-Works-with-Alexa-Google-Home-Siri-NS525E-L12-B3/319929479","frameId":"FD2E2C2E7E3F2C0209F63FC0F336E34C","hasUserGesture":false,"initiator":{"type":"other"},"loaderId":"DE57C21F725FBD63E35D90D54AEEF6C1","redirectHasExtraInfo":false,"request":{"headers":{"Upgrade-Insecure-Requests":"1","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36","sec-ch-ua":"\\"Chromium\\";v=\\"128\\", \\"Not;A=Brand\\";v=\\"24\\", \\"Google Chrome\\";v=\\"128\\"","sec-ch-ua-mobile":"?0","sec-ch-ua-

In [51]:
for log in logs:
    # 解析日志内容
    log_json = json.loads(log['message'])
    message = log_json['message']
    request_url = message['params']
    if 'documentURL' in request_url.keys():
        if 'graphql' in request_url['documentURL']:
            print(message)


{'method': 'Network.requestWillBeSent', 'params': {'documentURL': 'https://apionline.homedepot.com/federation-gateway/graphql?opname=productClientOnlyProduct', 'hasUserGesture': False, 'initiator': {'requestId': '3388.207', 'type': 'preflight', 'url': 'https://apionline.homedepot.com/federation-gateway/graphql?opname=productClientOnlyProduct'}, 'loaderId': '', 'redirectHasExtraInfo': False, 'request': {'headers': {'Accept': '*/*', 'Access-Control-Request-Headers': 'content-type,x-api-cookies,x-cloud-trace-context,x-current-url,x-debug,x-experience-name,x-hd-dc,x-parent-trace-id,x-thd-customer-token', 'Access-Control-Request-Method': 'POST', 'Origin': 'https://www.homedepot.com', 'Sec-Fetch-Mode': 'cors', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36'}, 'initialPriority': 'High', 'method': 'OPTIONS', 'referrerPolicy': 'strict-origin-when-cross-origin', 'url': 'https://apionline.homedepot.com/federat

In [32]:
log_json

{'message': {'method': 'Network.loadingFinished',
  'params': {'encodedDataLength': 227,
   'requestId': '33840.355',
   'timestamp': 30171.023144}},
 'webview': '63F249E1443F65A8BAD74F09A55FE6E0'}

In [25]:
def get_post_request_log(logs):
    for log in logs:
        # 解析日志内容
        log_json = json.loads(log['message'])
        message = log_json['message']
        
        # 查找 POST 请求
        if 'Network.requestWillBeSent' in message['method']:
            request_info = message['params']['request']
            if request_info['method'] == 'POST' and 'graphql?opname=recs' in request_info['url']:
                return request_info  # 返回 POST 请求

    return None  # 如果没有找到 POST 请求则返回 None

In [15]:
cookies, recs_request_log = get_cookies_via_selenium(url)

In [17]:
recs_request_log

{'level': 'INFO',
 'message': '{"message":{"method":"Network.requestWillBeSent","params":{"documentURL":"https://apionline.homedepot.com/federation-gateway/graphql?opname=recs","hasUserGesture":false,"initiator":{"requestId":"28276.298","type":"preflight","url":"https://apionline.homedepot.com/federation-gateway/graphql?opname=recs"},"loaderId":"","redirectHasExtraInfo":false,"request":{"headers":{"Accept":"*/*","Access-Control-Request-Headers":"content-type,x-api-cookies,x-cloud-trace-context,x-current-url,x-debug,x-experience-name,x-hd-dc,x-parent-trace-id,x-thd-customer-token","Access-Control-Request-Method":"POST","Origin":"https://www.homedepot.com","Sec-Fetch-Mode":"cors","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36"},"initialPriority":"High","method":"OPTIONS","referrerPolicy":"strict-origin-when-cross-origin","url":"https://apionline.homedepot.com/federation-gateway/graphql?opname=recs"},"r

In [18]:
headers, payload = extract_request_details(recs_request_log)

In [21]:
headers

{'Accept': '*/*',
 'Access-Control-Request-Headers': 'content-type,x-api-cookies,x-cloud-trace-context,x-current-url,x-debug,x-experience-name,x-hd-dc,x-parent-trace-id,x-thd-customer-token',
 'Access-Control-Request-Method': 'POST',
 'Origin': 'https://www.homedepot.com',
 'Sec-Fetch-Mode': 'cors',
 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36'}

In [22]:
payload

{}